In [1]:
import pandas as pd
import yaml
import numpy as np

# Given YAML data
yaml_data = """
compute-config:
  - name: "AISONLINE" 
    count: 5
    target:
      - SG
      - HK
    target-zone: dmz3
    os: SCB-RHEL8
    cpu: 32
    memory: 64
    san:
      - size: 200 # Storage size in GB 
        index: 1
        scsi-controller: 0
        mount-path: "/app/instancel" 
      - size: 700 # Storage size in GB
        index: 1
        scsi-controller: 0
        mount-path: "/app/batch"
  - name: "BATCH1"
    count: 7
    target:
      - SG
      - HK
    target-zone: dmz3
    OS: SCB-RHEL8
    cpu: 8
    memory: 32
    san:
      - size: 200 # Storage size in GB 
        index: 2
        scsi-controller: 0
        mount-path: "/app/instance2"
      - size: 10 # Storage size in GB 
        index: 2
        scsi-controller: 0
        mount-path: "/u01/app/oracle"
  - name: "ZZZtemp" 
    count: 5
    target:
      - SG
      - HK
    target-zone: dmz3
    os: SCB-RHEL8
    cpu: 64
    memory: 256
    nas:
      - nas-size: 200 # Storage size in GB 
        nas-index: 1
        nas-scsi-controller: 0
        nas-mount-path: "/app/instancel" 
      - nas-size: 700 # Storage size in GB
        nas-index: 1
        nas-scsi-controller: 0
        nas-mount-path: "/app/batch"
"""

# Parse YAML data
parsed_data = yaml.safe_load(yaml_data)

# Initialize lists to store data
non_nested_rows = []
nested_rows = []
categories = set()

# Iterate through each section under compute-config
for item in parsed_data['compute-config']:
    non_nested_row = {}
    for key, value in item.items():
        if isinstance(value, list):
            if all(isinstance(elem, dict) for elem in value):
                # Nested elements
                for i, elem in enumerate(value):
                    category = f"{key}_{i + 1}"
                    categories.add(category)
                    row = {'name': item['name'], 'Base_Category' : key , 'Category': category}
                    for k, v in elem.items():
                        row[k] = v  # Store nested data under the same column names
                    nested_rows.append(row)
            else:
                # Non-nested elements with multiple values
                non_nested_row[key] = "["+', '.join(str(v) for v in value)+"]"
        else:
            # Single value elements
            non_nested_row[key] = value
    
    non_nested_rows.append(non_nested_row)

# Create DataFrame for non-nested data
df_non_nested = pd.DataFrame(non_nested_rows)

# Sort categories alphabetically for consistent order
categories = sorted(list(categories))

# Create DataFrame for nested data
df_nested = pd.DataFrame(nested_rows)


In [22]:
import pandas as pd
import xlsxwriter

# Function to write sections to Excel
def write_sections(writer, df, unique_values):
    for value in unique_values:
        filtered_df = df[df['Base_Category'] == value]
        filtered_df = filtered_df.dropna(axis=1, how='all')
        
        filtered_df.to_excel(writer, sheet_name='Combined', startrow=writer.sheets['Combined'].__dict__.get('total_rows', 0), index=False)
        writer.sheets['Combined'].__dict__['total_rows'] = writer.sheets['Combined'].__dict__.get('total_rows', 0) + len(filtered_df) + 5
        
        # Add 5 empty rows for separation
        pd.DataFrame(index=range(5)).to_excel(writer, sheet_name='Combined', startrow=writer.sheets['Combined'].__dict__.get('total_rows', 0), header=False, index=False)
        writer.sheets['Combined'].__dict__['total_rows'] = writer.sheets['Combined'].__dict__.get('total_rows', 0) + 5

# Create a writer object
with pd.ExcelWriter('D:\\Y_output.xlsx', engine='xlsxwriter') as writer:
    # Write df_non_nested to Excel
    df_non_nested.to_excel(writer, sheet_name='Combined', index=False)
    
    # Get the row count of df_non_nested
    df_non_nested_rows = len(df_non_nested)
    
    # Insert 5 empty rows after df_non_nested
    pd.DataFrame(index=range(5)).to_excel(writer, sheet_name='Combined', startrow=df_non_nested_rows + 1, header=False, index=False)
    
    # Get unique values of Base_Category from df_nested
    unique_values_nested = df_nested['Base_Category'].unique()
    
    # Write nested sections for each unique Base_Category with formatting
    worksheet = writer.sheets['Combined']
    current_row = df_non_nested_rows + 7  # Start after the added empty rows
    
    for value in unique_values_nested:
        # Write "Data Content" text for each unique Base_Category
        worksheet.write(current_row, 0, f"Data Content :: {value}", writer.book.add_format({'bold': True}))
        current_row += 2  # Move to next row for nested data
        
        # Filter and write nested data for each unique Base_Category
        filtered_df = df_nested[df_nested['Base_Category'] == value]
        filtered_df = filtered_df.dropna(axis=1, how='all')
        
        filtered_df.to_excel(writer, sheet_name='Combined', startrow=current_row, index=False)
        current_row += len(filtered_df) + 5  # Move to the next section after 5 empty rows
    
    # Adjust the total_rows for correct tracking in the Excel file
    writer.sheets['Combined'].__dict__['total_rows'] = current_row



In [4]:
df_non_nested

,name,count,target,target-zone,os,cpu,memory,OS
0,AISONLINE,5,"SG, HK",dmz3,SCB-RHEL8,32,64,NaN
1,BATCH1,7,"SG, HK",dmz3,NaN,8,32,SCB-RHEL8
2,ZZZtemp,5,"SG, HK",dmz3,SCB-RHEL8,64,256,NaN


In [5]:
df_nested

,name,Base_Category,Category,size,index,scsi-controller,mount-path,nas-size,nas-index,nas-scsi-controller,nas-mount-path
0,AISONLINE,san,san_1,200.0,1.0,0.0,/app/instancel,NaN,NaN,NaN,NaN
1,AISONLINE,san,san_2,700.0,1.0,0.0,/app/batch,NaN,NaN,NaN,NaN
2,BATCH1,san,san_1,200.0,2.0,0.0,/app/instance2,NaN,NaN,NaN,NaN
3,BATCH1,san,san_2,10.0,2.0,0.0,/u01/app/oracle,NaN,NaN,NaN,NaN
4,ZZZtemp,nas,nas_1,NaN,NaN,NaN,NaN,200.0,1.0,0.0,/app/instancel
5,ZZZtemp,nas,nas_2,NaN,NaN,NaN,NaN,700.0,1.0,0.0,/app/batch
